# 聚类网页内容，用于推荐系统

## 读取mysql数据库，获取网页

In [1]:
import pymysql
cnx = pymysql.connect(host='localhost', user='root', password='123qwe12')
cursor = cnx.cursor()
cnx.select_db('IR_db')

In [2]:
def get_data_from_table(table_name):
    cursor.execute(f"SELECT id, title, content FROM {table_name}")
    results = cursor.fetchall()
    data = []
    for row in results:
        data.append({
            'id': row[0],
            'title': row[1],
            'content': row[2],
            'type': 'douban'
        })
    return data


def get_ids_from_same_title():
    # 创建SQL查询
    sql = "SELECT page_id FROM same_title"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_same_title = [result[0] for result in results]
    return ids_same_title


# 有问题
def get_page():
    cursor.execute("SELECT id, title, content FROM page")
    results = cursor.fetchall()
    # 获取相同title的id
    ids_same_title = get_ids_from_same_title()
    data = []
    for row in results:
        data.append({
            'id': row[0],
            'title': row[1],
            'content': row[2],
            'type': 'page'
        })
    # 从same_title中按title分组，获取每组的第一个id
    sql = "SELECT MIN(page_id) FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    ids = [result[0] for result in results]
    print(len(ids))
    # 找到这些id对应的url
    for id in ids:
        sql = "SELECT id, title, content FROM page WHERE id = %d" % id
        cursor.execute(sql)
        results = cursor.fetchall()
        data.append({
                'id': row[0],
                'title': row[1],
                'content': row[2],
                'type': 'page'
            })
    return data


In [3]:
data = get_data_from_table('douban')
data += get_data_from_table('html')
data += get_data_from_table('page')
print(len(data))

16151


## 对网页内容进行聚类

In [4]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
import numpy as np
import random

def random_substring(s, length=50):
    if len(s) <= length:
        return s
    start = random.randint(0, len(s) - length)
    return s[start:start+length]

with open('cn_stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = [line.strip() for line in f.readlines()]
stop_words += ["的", "了", "在", "是", "我", "有", "和", "就", "不", "人", "他", "这", "中", "大", "以", "到", "说", "等", "能", "也", "上", "或", "之", "但", "个", "都", "而", "啊", "把", "那", "你", "一", "为", "所", "年", "没", "着", "要", "与"]
stop_words = list(set(stop_words))

# 使用jieba进行分词
contents = [' '.join(word for word in jieba.cut(item['title']+random_substring(item['content'])) 
                     if word not in stop_words) for item in data]
print('cut word finished')

# 使用TF-IDF模型将文本转换为数值向量
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(contents)
print('tf-idf finished')
print(X.shape)

# # 使用PCA进行降维
# pca = PCA(n_components=10000)
# X_pca = pca.fit_transform(X.toarray())  # 转换为数组并降维
# print('PCA finished')

# 使用KMeans进行聚类
kmeans = KMeans(n_clusters=20)
kmeans.fit(X)
print('kmeans finished')

# 输出每个数据点的聚类标签
labels = kmeans.labels_
# 统计每个类的个数
counts = np.bincount(labels)
for i, count in enumerate(counts):
    print(f"Cluster {i} has {count} data points")

Building prefix dict from d:\pythonFile\python3.7\lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Projects\Temp\jieba.cache
Loading model cost 0.8924434185028076 seconds.
Prefix dict has been built succesfully.


cut word finished
tf-idf finished
(16151, 50261)
kmeans finished
Cluster 0 has 6665 data points
Cluster 1 has 318 data points
Cluster 2 has 1011 data points
Cluster 3 has 130 data points
Cluster 4 has 218 data points
Cluster 5 has 136 data points
Cluster 6 has 177 data points
Cluster 7 has 272 data points
Cluster 8 has 172 data points
Cluster 9 has 3383 data points
Cluster 10 has 359 data points
Cluster 11 has 147 data points
Cluster 12 has 436 data points
Cluster 13 has 1049 data points
Cluster 14 has 178 data points
Cluster 15 has 211 data points
Cluster 16 has 85 data points
Cluster 17 has 216 data points
Cluster 18 has 646 data points
Cluster 19 has 342 data points


## 保存聚类结果

In [5]:
# 创建表
cursor.execute("""
    CREATE TABLE IF NOT EXISTS cluster (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        type VARCHAR(255),
        web_id INT,
        label INT
    )
""")
# labels是KMeans的结果，vectors是TF-IDF向量
for item, label in zip(data, labels):
    # 插入数据
    cursor.execute("""
        INSERT INTO cluster (title, type, web_id, label)
        VALUES (%s, %s, %s, %s)
    """, (item['title'], item['type'], item['id'], int(label)))

# 提交
cnx.commit()

In [6]:
import joblib

# 保存模型
joblib.dump(kmeans, 'kmeans_model.pkl')
joblib.dump(vectorizer, 'tfidf_model.pkl')

['tfidf_model.pkl']

## 提取模型进行预测

In [7]:
kmeans = joblib.load('kmeans_model.pkl')
vectorizer = joblib.load('tfidf_model.pkl')

In [8]:
query = '新冠病毒'
# 使用jieba进行分词
query_cut = ' '.join(jieba.cut(query))
# 使用TF-IDF模型将文本转换为数值向量
query_vec = vectorizer.transform([query_cut])
# 使用KMeans进行预测
query_label = kmeans.predict(query_vec)
print(f"The query is in cluster {query_label[0]}")

The query is in cluster 9


### 获取对应类别下所有记录

In [9]:
# 查询数据库
cursor.execute("""
    SELECT web_id, title
    FROM cluster
    WHERE label = %s
""", (query_label[0],))

# 获取查询结果
results = cursor.fetchall()

# 打印结果
for row in results:
    print(f"web_id: {row[0]}, title: {row[1]}")

web_id: 82, title: On “3 Idiots”
web_id: 105, title: 闻香识女人 Scent of a Woman
web_id: 109, title: 印度人民的感情被伤害了
web_id: 115, title: 需要夢想又害怕夢想
web_id: 170, title: 《熔炉》始末
web_id: 173, title: 《熔炉》烘烤集体的良知
web_id: 177, title: 熔炉 도가니
web_id: 179, title: 这个清醒的小男孩，太让人心疼了
web_id: 192, title: 《寄生虫》背后的韩国寄生史
web_id: 197, title: 想想还是要参加一下影评大赛...
web_id: 245, title: 宏观企业管理，微观组织行为学
web_id: 246, title: 民主制度简明教程
web_id: 248, title: 12 Angry Men，令人惊叹并为之折服
web_id: 252, title: 十二怒汉 12 Angry Men
web_id: 261, title: 《辩护人》：我永远站在鸡蛋这一边
web_id: 264, title: 姜文的十个为什么 / 赵为民
web_id: 478, title: 荒岛余生 Cast Away
web_id: 494, title: [影评]幸福终点站：四大叔的全球化
web_id: 550, title: Buono giorno, Principessa!
web_id: 561, title: 《七号房的礼物》
web_id: 581, title: 作为父亲的愤怒--观《素媛》有感
web_id: 594, title: 現實不等於真相
web_id: 605, title: 《恐怖直播》：影帝级表演，个人私欲的绝对至上
web_id: 611, title: 要句道歉那么难
web_id: 623, title: 猫鼠游戏 Catch Me If You Can
web_id: 629, title: 除了钱，我们什么都有。
web_id: 708, title: 《钢琴家》的犹太华沙
web_id: 737, title: 所有的《敦刻尔克》花絮（still keep on updating...）


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import jieba.analyse

# 获取所有的title
titles = [row[1] for row in results]

# 使用jieba进行分词
titles = [' '.join(jieba.cut(title)) for title in titles]

# 使用TF-IDF模型将query和titles转换为向量
X = vectorizer.transform([query_cut] + titles)

# 计算query和每个title的余弦相似度
similarities = cosine_similarity(X[0:1], X[1:]).flatten()

# 获取最相似的5个title的索引
top5_indices = similarities.argsort()[-5:]

# 打印最相似的5个title
for index in top5_indices:
    title = results[index][1]
    keywords = jieba.analyse.extract_tags(title, topK=5)
    print(f"web_id: {results[index][0]}, title: {title}, keywords: {keywords}, similarity: {similarities[index]}")

web_id: 327, title: 新冠肺炎表述淡化-官方文件屡现新冠病毒感染者, keywords: ['新冠', '屡现', '病毒感染者', '肺炎', '淡化'], similarity: 0.33798392358706064
web_id: 249, title: 自然选出2022十大科学人物-北京大学曹云龙因追踪新冠病毒演化, keywords: ['2022', '新冠', '云龙', '追踪', '选出'], similarity: 0.3611543820399443
web_id: 227, title: 《血站新冠病毒感染防控工作指引（第二版）》印发，删除密接者、次密接者等暂缓献血有关内容综合国际做法和我国情况，明确感染新冠病毒(重型和危重型除外)，最后一次新冠病毒核酸检测或抗原检测阳性结果7天后可以献血；重型或危重型感染者，康复6个月后可以献血。, keywords: ['新冠', '献血', '密接', '危重', '重型'], similarity: 0.37609169896711814
web_id: 1443, title: 8月30日起-来华人员入境前无需新冠病毒核酸或抗原检测, keywords: ['30', '日起', '新冠', '入境', '核酸'], similarity: 0.41328025730921203
web_id: 67, title: 新冠病毒肺炎改名新冠病毒感染-从乙类甲管调整为乙类乙管, keywords: ['新冠', '乙类', '甲管', '乙管', '病毒感染'], similarity: 0.4364938559687719
